# Extracting text from PDF files

In [5]:
# %pip install PyPDF2 
# %pip install pdfminer.six
# %pip install pdfplumber
# %pip install pdf2image
# %pip install Pillow
# %pip install tesseract 
# %pip install pytesseract
# %pip install PyMuPDF
#%pip install fitz
# %pip install openpyxl
# %pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 6.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [82]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To perform OCR to extract text from images 
import pytesseract 
# To remove the additional created files
import os

In [83]:
# Create function to extract text

def text_extraction(element):
    # Extracting the text from the in line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))
    
    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [84]:
# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]
    
    return table

# Convert table into appropriate format
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapped texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string 
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string

# Create a function to check if the element is in any tables present in the page
def is_element_inside_any_table(element, page ,tables):
    x0, y0up, x1, y1up = element.bbox
    # Change the cordinates because the pdfminer counts from the botton to top of the page
    y0 = page.bbox[3] - y1up
    y1 = page.bbox[3] - y0up
    for table in tables:
        tx0, ty0, tx1, ty1 = table.bbox
        if tx0 <= x0 <= x1 <= tx1 and ty0 <= y0 <= y1 <= ty1:
            return True
    return False

# Function to find the table for a given element
def find_table_for_element(element, page ,tables):
    x0, y0up, x1, y1up = element.bbox
    # Change the cordinates because the pdfminer counts from the botton to top of the page
    y0 = page.bbox[3] - y1up
    y1 = page.bbox[3] - y0up
    for i, table in enumerate(tables):
        tx0, ty0, tx1, ty1 = table.bbox
        if tx0 <= x0 <= x1 <= tx1 and ty0 <= y0 <= y1 <= ty1:
            return i  # Return the index of the table
    return None  

In [85]:
# Create a function to crop the image elements from PDFs
def crop_image(element, pageObj):
    # Get the coordinates to crop the image from PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    # Crop the page using coordinates (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Save the cropped page to a new PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Save the cropped PDF to a new file
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Create a function to convert the PDF to images
def convert_to_images(input_file,poppler_path=r'/usr/local/bin/poppler-23.11.0'):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = './PDF_image.png'
    image.save(output_file, 'PNG')

# Create a function to read text from images
def image_to_text(image_path):
    # Read the image
    img = Image.open(image_path)
    # Extract the text from the image
    text = pytesseract.image_to_string(img)
    return text

In [86]:
# Find the PDF path
pdf_path = '/Users/macbook/Desktop/Job/OVGU/a.pdf'

# Create a pdf file object
pdfFileObj = open(pdf_path, 'rb')
# Create a pdf reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

In [88]:
# Create the dictionary to extract text from each image
text_per_page = {}
# Create a boolean variable for image detection
image_flag = False
# We extract the pages from the PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    try: # Initialize the variables needed for the text extraction from the page
        pageObj = pdfReaded.pages[pagenum]
        dictionary = pdfReaded.get_form_text_fields()
        page_text = []
        line_format = []
        text_from_images = []
        text_from_tables = []
        page_content = []
        # Initialize the number of the examined tables
        table_in_page= -1
        # Open the pdf file
        pdf = pdfplumber.open(pdf_path)
        # Find the examined page
        page_tables = pdf.pages[pagenum]
        # Find the number of tables in the page
        tables = page_tables.find_tables()
        if len(tables)!=0:
            table_in_page = 0

        # Extracting the tables of the page
        for table_num in range(len(tables)):
            # Extract the information of the table
            table = extract_table(pdf_path, pagenum, table_num)
            # Convert the table information in structured string format
            table_string = table_converter(table)
            # Append the table string into a list
            text_from_tables.append(table_string)

        # Find all the elements
        page_elements = [(element.y1, element) for element in page._objs]
        # Sort all the element as they appear in the page 
        page_elements.sort(key=lambda a: a[0], reverse=True)


        # Find the elements that composed a page
        for i,component in enumerate(page_elements):
            # Extract the element of the page layout
            element = component[1]

            # Check the elements for tables
            if table_in_page == -1:
                pass
            else:
                if is_element_inside_any_table(element, page ,tables):
                    table_found = find_table_for_element(element,page ,tables)
                    if table_found == table_in_page and table_found != None:    
                        page_content.append(text_from_tables[table_in_page])
                        page_text.append('table')
                        line_format.append('table')
                        table_in_page+=1
                    # Pass this iteration because the content of this element was extracted from the tables
                    continue

            if not is_element_inside_any_table(element,page,tables):

                # Check if the element is text element
                if isinstance(element, LTTextContainer):
                    # Use the function to extract the text and format for each text element
                    (line_text, format_per_line) = text_extraction(element)
                    # Append the text of each line to the page text
                    page_text.append(line_text)
                    # Append the format for each line containing text
                    line_format.append(format_per_line)
                    page_content.append(line_text)


                # Check the elements for images
                if isinstance(element, LTFigure):
                #if True:
                    # Crop the image from PDF
                    crop_image(element, pageObj)
                    # Convert the croped pdf to image
                    convert_to_images('cropped_image.pdf')
                    # Extract the text from image
                    image_text = image_to_text('PDF_image.png')
                    text_from_images.append(image_text)
                    page_content.append(image_text)
                    # Add a placeholder in the text and format lists
                    page_text.append('image')
                    line_format.append('image')
                    # Update the flag for image detection
                    image_flag = True
    except KeyboardInterrupt:
        # Handle KeyboardInterrupt (Ctrl+C)
        print("KeyboardInterrupt: Stopping execution gracefully.")
        # You can add cleanup operations here if needed.
        break  # Exit the loop
            


    # Create the key of the dictionary
    dctkey = 'Page_'+str(pagenum)
    # Add the list of list as value of the page key
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

In [89]:
# Close the pdf file object
pdfFileObj.close()

In [ ]:
# # Delete the additional files created if image is detected
# if image_flag:
#     os.remove('cropped_image.pdf')
#     os.remove('PDF_image.png')

In [ ]:
# Display the content of the page
result = ''.join(text_per_page['Page_0'][3])
print(result)

|Name, ggf. Geburtsname|Rufname|None|alle Vornamen|None|
|Geburtsdatum|Geburtsort|None|None|Staatsangehörigkeit|
|Anschrift Hauptwohnsitz: (Straße, Hausnr., bei Wohnheim Zimmernr., Postleitzahl, Wohnort)|None|None|None|None|
|Erreichbarkeit - Telefonnummer:|None|E-Mail-Adresse:|None|None|
|Familienstand: ☐ledig ☐verheiratet ☐geschieden ☐eingetragene Partnerschaft|None|None|None|Geschlecht: ☐männlich ☐ weiblich ☐ divers|
|Name Bank/ Kreditinstitut (kein Sparkonto, nur eigenes Konto):|None|None|None|Kontoinhaber (falls abweichend):|
|IBAN: DE|None|None|None|BIC:|


In [90]:
print(dictionary)

{'Randfeld_rot': None, 'Randfeld_grün': None, '09': ' Version 15082022', '101c': 'Syed Muhammad Hamza', '103': 'Stendaler Strasse 2, 39106, Magdeburg.', '101a': 'Zaidi', '101b': None, '102a': '30.08.1995', '102b': 'Karachi, Pakistan', '102c': 'Pakistani', '104a': '01794298717', '104b': 'syed1.zaidi@st.ovgu.de', '106a': 'Sparkasse', '106b': None, '107g': 'NOLADE21MDG', '107e': '6801', '107f': '84', '108': 'TK ', '107a': '06', '107b': '8105', '107c': '3272', '107d': '1631', '110a': '08', '110b': '300895', '110c': 'Z', '110d': '006', '112a': '74', '112b': '501', '112c': '223', '112d': '284', '113': None, '114': None, '115': None, '116TF': None, '116von': None, '116bis': None, '117': None, '202': None, '202von': None, '202bis': None, '205': 'Mcdonalds', '205TF2': '900', '205von': '17.07.23', '205bis': '30.06.24', '205TF3': '20', '207': None, '207von': None, '207bis': None, '207TF2': None, '207TF3': None, '210': None, '210TF1': None, '210TF2': None, '211': '30.10.2023'}


In [10]:
import sys
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1

pdf_path = '/Users/macbook/Desktop/Job/OVGU/Data/F02.pdf'
fp = open(pdf_path, 'rb')

field_mapping = {
    b'1.1': 'Nachname',
    b'1.2': 'Vorname',
    b'1.3': 'Geburtsdatum',
    b'1.4': 'Geschlecht',
    b'1.5': 'Straße',
    b'1.5NR': 'Hausnummer',
    b'1.6PLZ': 'PLZ',
    b'1.6': 'Ort',
    b'1.7': 'Telefon',
    b'1.8': 'E-Mail',
    b'1.9.1': 'Staatsangehörigkeit',
    b'1.10Einschreibung': 'Einschreibung',
    b'1.11': 'Besteht eine Behinderung',
    b'2.1': 'Welche art der Behinderung',
    b'2.2': 'Wann erwarben Sie die Behinderung',
    b'2.3.HZB': 'Wo erwarben Sie die Behinderung',
    b'2.3BL': 'Wo erwarben Sie die Behinderung?',
    b'2.4': 'Bitte geben Sie an, ob Sie eine der folgenden Leistungen beziehen:',
    b'3.1': 'Wo erfolgte die Hochschulzugangsberechtigung?',
    b'3.2': 'Bitte geben Sie das Kfz-Kennzeichen',
    b'3.3': 'Bezeichnung der Hochschuleinrichtung',
    b'3.4M': 'Monat',
    b'3.5J': 'Jahr',
    b'4.1': 'Zur Promotion berechtigende',
    b'4.2': 'Art des Abschlusses',
    b'4.3': 'In welchem Studiengang wurden Sie geprüft',
    b'4.4': 'Mit welcher Gesamtnote wurde die Prüfung beurteilt?',
    b'5.1': 'Fachgebiet',
    b'5.2': 'Art der Promotion',
    b'5.3': 'Art der Dissertation',
    b'5.4': 'Teilnahme an einem strukturierten Promotionsprogramm',
    b'5.6': 'Betreuer/in der Dissertation',
    b'6Ort': 'Ort',
    b'6Datum': 'Datum'
}

parser = PDFParser(fp)
doc = PDFDocument(parser)
fields = resolve1(doc.catalog['AcroForm'])['Fields']
for i in fields:
    field = resolve1(i)
    name, value = field.get('T'), field.get('V')
    if name and name in field_mapping:
        mapped_name = field_mapping[name]
        
        # Convert 'PSLiteral' to string
        if isinstance(value, str):
            value = value.strip("b'/'")  # Remove unnecessary characters
        else:
            value = str(value).strip("b'/") 
            
        print('{0} = {1}'.format(mapped_name, value))

Ort = Magdeburg
Datum = 15.06.2023
Nachname = Freudenberg
Besteht eine Behinderung = Ja
Einschreibung = Nein
Welche art der Behinderung = Abendgymn./Kolleg (aHR)
Wann erwarben Sie die Behinderung = 010788
Wo erwarben Sie die Behinderung = I
Wo erwarben Sie die Behinderung? = Sachsen-Anhalt
Bitte geben Sie an, ob Sie eine der folgenden Leistungen beziehen: = \xfe\xff\x00K\x00\xd6\x00T
Wo erfolgte die Hochschulzugangsberechtigung? = I
Bitte geben Sie das Kfz-Kennzeichen = B
Bezeichnung der Hochschuleinrichtung = \xfe\xff\x00H\x00o\x00c\x00h\x00s\x00c\x00h\x00u\x00l\x00e\x00 \x00f\x00\xfc\x00r\x00 \x00\xd6\x00k\x00o\x00n\x00o\x00m\x00i\x00e\x00 \x00B\x00e\x00r\x00l\x00i\x00n
Monat = 09
Zur Promotion berechtigende = Ja
Art des Abschlusses = Diplom
In welchem Studiengang wurden Sie geprüft = Informatik
Mit welcher Gesamtnote wurde die Prüfung beurteilt? = 1,4
Fachgebiet = Informatik
Teilnahme an einem strukturierten Promotionsprogramm = Nein
Art der Dissertation = Monografie
Vorname = Rita 

In [9]:
import pandas as pd
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1
import chardet
import openpyxl


def extract_data(pdf_path):
    field_mapping = {
        b'1.1': 'Nachname',
        b'1.2': 'Vorname',
        b'1.3': 'Geburtsdatum',
        b'1.4': 'Geschlecht',
        b'1.5': 'Straße',
        b'1.5NR': 'Hausnummer',
        b'1.6PLZ': 'PLZ',
        b'1.6': 'Ort',
        b'1.7': 'Telefon',
        b'1.8': 'E-Mail',
        b'1.9.1': 'Staatsangehörigkeit',
        b'1.10Einschreibung': 'Einschreibung',
        b'1.11': 'Besteht eine Behinderung',
        b'2.1': 'Welche art der Behinderung',
        b'2.2': 'Wann erwarben Sie die Behinderung',
        b'2.3.HZB': 'Wo erwarben Sie die Behinderung',
        b'2.3BL': 'Wo erwarben Sie die Behinderung?',
        b'2.4': 'Bitte geben Sie an, ob Sie eine der folgenden Leistungen beziehen:',
        b'3.1': 'Wo erfolgte die Hochschulzugangsberechtigung?',
        b'3.2': 'Bitte geben Sie das Kfz-Kennzeichen',
        b'3.3': 'Bezeichnung der Hochschuleinrichtung',
        b'3.4M': 'Monat',
        b'3.5J': 'Jahr',
        b'4.1': 'Zur Promotion berechtigende',
        b'4.2': 'Art des Abschlusses',
        b'4.3': 'In welchem Studiengang wurden Sie geprüft',
        b'4.4': 'Mit welcher Gesamtnote wurde die Prüfung beurteilt?',
        b'5.1': 'Fachgebiet',
        b'5.2': 'Art der Promotion',
        b'5.3': 'Art der Dissertation',
        b'5.4': 'Teilnahme an einem strukturierten Promotionsprogramm',
        b'5.6': 'Betreuer/in der Dissertation',
        b'6Ort': 'Ort',
        b'6Datum': 'Datum'
    }

    data_dict = {}  

    parser = PDFParser(open(pdf_path, 'rb'))
    doc = PDFDocument(parser)
    fields = resolve1(doc.catalog['AcroForm'])['Fields']
    for i in fields:
        field = resolve1(i)
        name, value = field.get('T'), field.get('V')
        if name in field_mapping:
            mapped_name = field_mapping[name]

            if isinstance(value, bytes):
                result = chardet.detect(value)
                encoding = result["encoding"]
                value = value.decode(encoding, errors="replace").strip("'/")
                
            elif isinstance(value, str):
                value = value.strip("b'/")

            data_dict[mapped_name] = value


    df = pd.DataFrame([data_dict])
    df = df[[col for col in field_mapping.values() if col in df.columns]]

    output_excel_path = pdf_path.replace('.pdf', '_output.xlsx')
    df.to_excel(output_excel_path, index=False)
    print(f"Data extracted from the PDF has been saved to {output_excel_path}")
    print(df)

pdf_file_path = '/Users/macbook/Desktop/Job/OVGU/Data/F02.pdf'
extract_data(pdf_file_path)


Data extracted from the PDF has been saved to /Users/macbook/Desktop/Job/OVGU/Data/F02_output.xlsx
      Nachname      Vorname Geburtsdatum Geschlecht          Straße  \
0  Freudenberg  Rita Ingrid     25041968          W  Potsdamer Str.   

  Hausnummer    PLZ        Ort       Telefon                    E-Mail  ...  \
0         3a  39114  Magdeburg  0170 8321836  rita.freudenberg@ovgu.de  ...   

  Zur Promotion berechtigende Art des Abschlusses  \
0                       /'Ja'              Diplom   

  In welchem Studiengang wurden Sie geprüft  \
0                                Informatik   

  Mit welcher Gesamtnote wurde die Prüfung beurteilt?  Fachgebiet  \
0                                                1,4   Informatik   

  Art der Dissertation Teilnahme an einem strukturierten Promotionsprogramm  \
0        /'Monografie'                                            /'Nein'     

  Betreuer/in der Dissertation        Ort       Datum  
0     Prof. Andreas Nürnberger  Magdeburg  

In [ ]:
import fitz  

def extract_form_fields(pdf_path):
    doc = fitz.open(pdf_path)
    form_fields = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        widgets = page.widgets()  # widgets are the form fields

        for widget in widgets:  # Iterate through fields
            field_name = widget.field_name
            field_value = widget.field_value
            if field_name:  # Ensure the widget has a name
                form_fields[field_name] = field_value

    doc.close()
    return form_fields

pdf_path = '/Users/macbook/Desktop/Job/OVGU/Data/F02.pdf'
form_fields = extract_form_fields(pdf_path)  # Call the function and assign the result to form_fields
print("Extracted Form Fields:")
for name, value in form_fields.items():
    print(f"{name}: {value}")


In [27]:
import fitz  

def extract_form_fields(pdf_path, field_name_mapping):

    doc = fitz.open(pdf_path)
    form_fields = {}

    for page_num in range(len(doc)): 
        page = doc.load_page(page_num) 
        widgets = page.widgets() # widgets are the form fields

        for widget in widgets:  # Iterate through fields
            field_name = widget.field_name
            field_value = widget.field_value
            descriptive_name = field_name_mapping.get(field_name)
            if field_name:  # Ensure the widget has a name
                form_fields[descriptive_name] = field_value

    doc.close()  
    return form_fields

field_name_mapping = {
    '1.1': 'Nachname',
    '1.2': 'Vorname',
    '1.3': 'Geburtsdatum',
    '1.4': 'Geschlecht',
    '1.5': 'Straße',
    '1.5NR': 'Hausnummer',
    '1.6PLZ': 'PLZ',
    '1.6': 'Ort',
    '1.7': 'Telefon',
    '1.8': 'E-Mail',
    '1.9.1': 'Staatsangehörigkeit(1)',
    '1.9.2': 'Staatsangehörigkeit(2)',
    '1.10Einschreibung': 'Einschreibung',
    '1.11': 'Besteht eine Behinderung',
    '2.1': 'Welche art der Behinderung',
    '2.2': 'Wann erwarben Sie die Behinderung',
    '2.3.HZB': 'Wo erwarben Sie die Behinderung',
    '2.3BL': 'Wo erwarben Sie die Behinderung?',
    '2.4': 'Bitte geben Sie an, ob Sie eine der folgenden Leistungen beziehen:',
    '3.1': 'Wo erfolgte die Hochschulzugangsberechtigung?',
    '3.2': 'Bitte geben Sie das Kfz-Kennzeichen',
    '3.3': 'Bezeichnung der Hochschuleinrichtung',
    '3.4M': 'Monat',
    '3.5J': 'Jahr',
    '4.1': 'Zur Promotion berechtigende',
    '4.2': 'Art des Abschlusses',
    '4.3': 'In welchem Studiengang wurden Sie geprüft',
    '4.4': 'Mit welcher Gesamtnote wurde die Prüfung beurteilt?',
    '4.5': 'Wann wurde das Prüfungsergebnis offiziell festgestellt?',
    '4.6': 'Wo erwarben Sie den Hochschulabschluss?',
    '4.7': 'Bitte geben Sie das Kfz-Kennzeichen des Erwerbsortes an?',
    '4.8': 'Bezeichnung der Hochschuleinrichtung',
    '5.1': 'Fachgebiet',
    '5.2': 'Art der Promotion',
    '5.3': 'Art der Dissertation',
    '5.4': 'Teilnahme an einem strukturierten Promotionsprogramm',
    '5.5': 'Thema der Dissertation',
    '5.6': 'Betreuer/in der Dissertation',
    '6Ort': 'Ort',
    '6Datum': 'Datum'
}


pdf_path = '/Users/macbook/Desktop/Job/OVGU/Data/F02.pdf' 
form_fields = extract_form_fields(pdf_path, field_name_mapping) 
print("Extracted Form Fields:")
for name, value in form_fields.items():
    print(f"{name}: {value}")


Extracted Form Fields:
None:  Version 18052017
Nachname: Freudenberg
Vorname: Rita Ingrid
Geburtsdatum: 25041968
Geschlecht: W
Straße: Potsdamer Str.
Hausnummer: 3a
PLZ: 39114
Ort: Magdeburg
Telefon: 0170 8321836
E-Mail: rita.freudenberg@ovgu.de
Staatsangehörigkeit(1): D
Staatsangehörigkeit(2): 
Einschreibung: Nein
Besteht eine Behinderung: Ja
Welche art der Behinderung: Abendgymn./Kolleg (aHR)
Wann erwarben Sie die Behinderung: 010788
Wo erwarben Sie die Behinderung: I
Wo erwarben Sie die Behinderung?: Sachsen-Anhalt
Bitte geben Sie an, ob Sie eine der folgenden Leistungen beziehen:: KÖT
Wo erfolgte die Hochschulzugangsberechtigung?: I
Bitte geben Sie das Kfz-Kennzeichen: B
Bezeichnung der Hochschuleinrichtung: Otto-von-Guericke-Universität
Monat: 09
Zur Promotion berechtigende: Ja
Art des Abschlusses: Diplom
In welchem Studiengang wurden Sie geprüft: Informatik
Mit welcher Gesamtnote wurde die Prüfung beurteilt?: 1,4
Wo erwarben Sie den Hochschulabschluss?: I
Bitte geben Sie das Kfz-

In [40]:
import fitz  
import pandas as pd  

pdf_path = '/Users/macbook/Desktop/Job/OVGU/Data/F02.pdf'

def extract_data_and_save(pdf_file_path, field_name_mapping):
    doc = fitz.open(pdf_file_path)
    data_dict = {}


    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        widgets = page.widgets()

        for widget in widgets:
            field_name = widget.field_name
            field_value = widget.field_value
            if field_name and field_value:  
                data_dict[field_name] = field_value

    doc.close()

    filtered_data = {field_name_mapping.get(k): v for k, v in data_dict.items() if k in field_name_mapping and v}

    if filtered_data:  
        df = pd.DataFrame([filtered_data])
    else:
        print("No valid data found to include in the DataFrame.")
        return

    output_excel_path = pdf_file_path.replace('.pdf', '_fitz_f02output.xlsx')    
    df.to_excel(output_excel_path, index=False)
    
    print(f"Data extracted from the PDF has been saved to {output_excel_path}")
    print(df)


field_name_mapping = {
    '1.1': 'Nachname',
    '1.2': 'Vorname',
    '1.3': 'Geburtsdatum',
    '1.4': 'Geschlecht',
    '1.5': 'Straße',
    '1.5NR': 'Hausnummer',
    '1.6PLZ': 'PLZ',
    '1.6': 'Ort',
    '1.7': 'Telefon',
    '1.8': 'E-Mail',
    '1.9.1': 'Staatsangehörigkeit(1)',
    '1.9.2': 'Staatsangehörigkeit(2)',
    '1.10Einschreibung': 'Einschreibung',
    '1.11': 'Besteht eine Behinderung',
    '2.1': 'Welche art der Behinderung',
    '2.2': 'Wann erwarben Sie die Behinderung',
    '2.3.HZB': 'Wo erwarben Sie die Behinderung',
    '2.3BL': 'Wo erwarben Sie die Behinderung?',
    '2.4': 'Bitte geben Sie an, ob Sie eine der folgenden Leistungen beziehen:',
    '3.1': 'Wo erfolgte die Hochschulzugangsberechtigung?',
    '3.2': 'Bitte geben Sie das Kfz-Kennzeichen',
    '3.3': 'Bezeichnung der Hochschuleinrichtung',
    '3.4M': 'Monat',
    '3.5J': 'Jahr',
    '4.1': 'Zur Promotion berechtigende',
    '4.2': 'Art des Abschlusses',
    '4.3': 'In welchem Studiengang wurden Sie geprüft',
    '4.4': 'Mit welcher Gesamtnote wurde die Prüfung beurteilt?',
    '4.5': 'Wann wurde das Prüfungsergebnis offiziell festgestellt?',
    '4.6': 'Wo erwarben Sie den Hochschulabschluss?',
    '4.7': 'Bitte geben Sie das Kfz-Kennzeichen des Erwerbsortes an?',
    '4.8': 'Bezeichnung der Hochschuleinrichtung',
    '5.1': 'Fachgebiet',
    '5.2': 'Art der Promotion',
    '5.3': 'Art der Dissertation',
    '5.4': 'Teilnahme an einem strukturierten Promotionsprogramm',
    '5.5': 'Thema der Dissertation',
    '5.6': 'Betreuer/in der Dissertation',
    '6Ort': 'Ort',
    '6Datum': 'Datum'
}

extract_data_and_save(pdf_file_path, field_name_mapping)


Data extracted from the PDF has been saved to /Users/macbook/Desktop/Job/OVGU/Data/F02_fitz_f06output.xlsx
      Nachname      Vorname Geburtsdatum Geschlecht          Straße  \
0  Freudenberg  Rita Ingrid     25041968          W  Potsdamer Str.   

  Hausnummer    PLZ        Ort       Telefon                    E-Mail  ...  \
0         3a  39114  Magdeburg  0170 8321836  rita.freudenberg@ovgu.de  ...   

  In welchem Studiengang wurden Sie geprüft  \
0                                Informatik   

  Mit welcher Gesamtnote wurde die Prüfung beurteilt?  \
0                                                1,4    

  Wo erwarben Sie den Hochschulabschluss?  \
0                                       I   

  Bitte geben Sie das Kfz-Kennzeichen des Erwerbsortes an?  \
0                                                 MD         

  Wann wurde das Prüfungsergebnis offiziell festgestellt?  Fachgebiet  \
0                                             151097        Informatik   

  Art der Disser

In [2]:
import fitz  
import pandas as pd  
import os  
import glob  

def extract_form_fields(pdf_file_path, field_name_mapping):
    doc = fitz.open(pdf_file_path)
    data_dict = {}
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        widgets = page.widgets()
        for widget in widgets:
            field_name = widget.field_name
            field_value = widget.field_value
            if field_name and field_value and field_name in field_name_mapping:
                data_dict[field_name_mapping[field_name]] = field_value
    doc.close()
    return data_dict

def process_pdf_folder(pdf_folder_path, field_name_mapping, output_excel_path):
    pdf_file_paths = glob.glob(os.path.join(pdf_folder_path, '*.pdf'))  
    aggregated_data = []
    for pdf_path in pdf_file_paths:
        extracted_data = extract_form_fields(pdf_path, field_name_mapping)
        if extracted_data:  
            aggregated_data.append(extracted_data)
    if aggregated_data:
        df = pd.DataFrame(aggregated_data)
        df.to_excel(output_excel_path, index=False)
        print(f"Data extracted from the PDFs has been saved to {output_excel_path}")
    else:
        print("No valid data found to include in the DataFrame.")


field_name_mapping = {
    '1.1': 'Nachname',
    '1.2': 'Vorname',
    '1.3': 'Geburtsdatum',
    '1.4': 'Geschlecht',
    '1.5': 'Straße',
    '1.5NR': 'Hausnummer',
    '1.6PLZ': 'PLZ',
    '1.6': 'Ort',
    '1.7': 'Telefon',
    '1.8': 'E-Mail',
    '1.9.1': 'Staatsangehörigkeit',
    '1.9.2': 'Weitere Staatsangehörigkeit',
    '1.10Einschreibung': 'Einschreibung als Promotionsstudierenden',
    '1.11': 'Beschäftigungsverhältnis mit OVGU / Klinikum',
    '2.1': 'Art der HZB',
    '2.2': 'Datum HZB-Erwerb',
    '2.3.HZB': 'Staat HZB-Erwerb',
    '2.3BL': 'Landkreis HZB-Erwerb',
    '2.4': 'Staat der Ersteinschreibung',
    '3.1': 'Wo erfolgte die Hochschulzugangsberechtigung',
    '3.2': 'Bitte geben Sie das Kfz-Kennzeichen',
    '3.3': 'Hochschule der Ersteinschreibung (falls Inland)',
    '3.4M': 'Monat',
    '3.4J': 'Jahr',
    '4.1': 'Zur Promotion berechtigende',
    '4.2': 'Art des Abschlusses',
    '4.3': 'In welchem Studiengang wurden Sie geprüft',
    '4.4': 'Mit welcher Gesamtnote wurde die Prüfung beurteilt?',
    '4.5': 'Wann wurde das Prüfungsergebnis offiziell festgestellt?',
    '4.6': 'Wo erwarben Sie den Hochschulabschluss?',
    '4.7': 'Bitte geben Sie das Kfz-Kennzeichen des Erwerbsortes an?',
    '4.8': 'Bezeichnung der Hochschuleinrichtung',
    '5.1': 'Fachgebiet der Promotion',
    'ArtProm': 'Art der Promotion',
    '5.3': 'Art der Dissertation',
    '5.4': 'Teilnahme an strukturiertem Promotionsprogramm',
    '5.5.Block': 'Thema',
    '5.6': 'Betreuer',
    '6Ort': 'Ort',
    '6Datum': 'Datum der Zulassung'
}

pdf_folder_path = '/Users/macbook/Desktop/Job/OVGU/Data/pdf_files'
output_excel_path = '/Users/macbook/Desktop/Job/OVGU/pdf_fields_extractor/pdfextractor/aggregated_output.xlsx'

process_pdf_folder(pdf_folder_path, field_name_mapping, output_excel_path)


Data extracted from the PDFs has been saved to /Users/macbook/Desktop/Job/OVGU/pdf_fields_extractor/pdfextractor/aggregated_output.xlsx


In [10]:
import fitz  # PyMuPDF
import pandas as pd  
import os  
import glob  

def extract_form_fields(pdf_file_path):
    doc = fitz.open(pdf_file_path)
    data_dict = {}
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        widgets = page.widgets()
        for widget in widgets:
            field_name = widget.field_name
            field_value = widget.field_value
            if field_name and field_value:
                data_dict[field_name] = field_value
    doc.close()
    return data_dict

def process_pdf_folder(pdf_folder_path, output_excel_path):
    pdf_file_paths = glob.glob(os.path.join(pdf_folder_path, '*.pdf'))  
    aggregated_data = []
    for pdf_path in pdf_file_paths:
        extracted_data = extract_form_fields(pdf_path)
        if extracted_data:  
            aggregated_data.append(extracted_data)
    if aggregated_data:
        df = pd.DataFrame(aggregated_data)
        df.to_excel(output_excel_path, index=False)
        print(f"Data extracted from the PDFs has been saved to {output_excel_path}")
    else:
        print("No valid data found to include in the DataFrame.")

# Example usage
pdf_folder_path = '/Users/macbook/Desktop/Job/OVGU/Data/pdf_files'
output_excel_path = '/Users/macbook/Desktop/Job/OVGU/pdf_fields_extractor/pdfextractor/fields_output.xlsx'

process_pdf_folder(pdf_folder_path, output_excel_path)


Data extracted from the PDFs has been saved to /Users/macbook/Desktop/Job/OVGU/pdf_fields_extractor/pdfextractor/fields_output.xlsx
